In [34]:
import os
import json
import re
from datetime import datetime
import hashlib

from langchain_openai import AzureOpenAIEmbeddings
from openai import OpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken
from langsmith import traceable
from langsmith.wrappers import wrap_openai
from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

# openai_client = AzureOpenAI(
#     api_key=os.getenv("AZURE_CHAT_OPENAI_API_KEY"), 
#     api_version=os.getenv("AZURE_CHAT_OPENAI_API_VERSION"), 
#     azure_endpoint=os.getenv("AZURE_CHAT_OPENAI_ENDPOINT")
#     )
openai_client = wrap_openai(OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
    ))

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

In [35]:
log_file_name = ""


def ensure_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)


def log(message):
    global log_file_name
    ensure_directory("logs")
    with open(log_file_name, "a") as file:
        file.write(f"{datetime.now()}\n{message}\n")

In [36]:
conversation_price = 0
conversation_ct = 0
conversation_pt = 0
conversation_tt = 0


def get_conversation_price(new_completion, model="gpt4o"):
    global conversation_price, conversation_ct, conversation_pt, conversation_tt
    USD_price_in_COP = 4100

    if model == "gpt4o-mini":
        in_price = 0.00000015
        out_price = 0.0000006
    if model == "gpt-4o-2024-08-06":
        in_price = 0.0000025
        out_price = 0.000010
    if model == "gpt4o":
        in_price = 0.000005
        out_price = 0.000015
    if model == "llama-3-8b":
        in_price = 0.00000005
        out_price = 0.00000008
    if model == "llama-3-70b":
        in_price = 0.00000059
        out_price = 0.0000007
    if model == "haiku":
        in_price = 0.00000025
        out_price = 0.00000125

    ct = new_completion.usage.completion_tokens
    pt = new_completion.usage.prompt_tokens
    usd_total_price = (ct * out_price) + (pt * in_price)
    cop_total_price = usd_total_price * USD_price_in_COP
    conversation_price += cop_total_price
    print(
        f"💰 Conversation price: ${conversation_price} COP\n"
        f"   This message: ${cop_total_price} COP "
        f""
    )
    log(
        f"💰 Conversation price: ${conversation_price} COP\n"
        f"   This message: ${cop_total_price} COP "
        f""
    )

    return

In [37]:
### 🔑 Definir system prompt y tools

messages = []
conversation = []
user_input = ""

codes_to_laws = """
- Código Penal: Ley 599 de 2000
- Código Civil: Ley 57 de 1887
- Código de Comercio: Decreto 410 de 1971
- Código de Procedimiento Civil: Decreto 1400 de 1970
- Código de Procedimiento Penal: Ley 906 de 2004
- Código General del Proceso: Ley 1564 de 2012
- Código de la Infancia y la Adolescencia: Ley 1098 de 2006
- Código Nacional de Policía: Ley 1801 de 2016
- Código de Recursos Naturales: Decreto 2811 de 1974
- Código Electoral: Decreto 2241 de 1986
- Código Disciplinario Único: Ley 734 de 2002
- Código Contencioso Administrativo: Ley 1437 de 2011 (anteriormente Decreto 1 de 1984)
- Código de Minas: Ley 685 de 2001
- Código de Educación: Ley 115 de 1994
- Código Nacional de Tránsito Terrestre: Ley 769 de 2002
- Código de Construcción del Distrito Capital de Bogotá: Acuerdo 20 de 1995
- Código de Construcción Sismo-Resistente: NSR-10 
- Código de Régimen Departamental: Decreto 1222 de 1986
- Código de Régimen Político y Municipal: Decreto-Ley 1333 de 1986
- Código Penal Militar: Ley 1407 de 2010
- Código Penitenciario y Carcelario: Ley 65 de 1993
- Código Procesal del Trabajo y del Seguro Social: Decreto-Ley 2158 de 1948
- Código Sustantivo del Trabajo: Decreto 2663 de 1950
- Código Sanitario Nacional: Ley 9 de 1979
- Código General Disciplinario: Ley 1952 de 2019
- Código Nacional de Seguridad y Convivencia Ciudadana: Ley 1801 de 2016
- Código Penal Militar: Ley 522 de 1999 """


get_next_chunk = {
    "type": "function",
    "function": {
        "name": "get_next_chunk",
        "description": "Use this tool to get when an important sources comes incomplete or the content is cut off. This tool will help you retrieve the following section of that given source.",
        "parameters": {
            "type": "object",
            "properties": {
                "source_id": {
                    "type": "string",
                    "description": "the unique identifier of the chunk that is incomplete. eg:'20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30'",
                },
            },
            "required": ["source_id"],
        },
    },
}

response_system_template = f"""
Eres Ariel, un asistente de investigación legal en derecho colombiano. Sigue estas instrucciones al responder: 
  
1. Consulta de Fuentes: 
- Tienes acceso a algunas fuentes que previamente has encontrado y seleccionado para generar esta respuesta. Dentro de esas fuentes se encuentra toda la información disponible para responder.  
- Expresa toda la información que encuentres en las fuentes proporcionadas y que sea relevante para responder. 
- Si las fuentes no ayudan a responder la pregunta, responde: "No encuentro información con esos términos, ¿puedes reformular tu consulta?" 
- Las fuentes vienen en fragmentos del documento completo con los siguientes campos: 
    - id: Identificador único del fragmento 
    - title: Título del documento 
    - author: Autor del documento 
    - keywords: Tema legal, (puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros). 
    - category: Tipo de documento legal. (Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralegal y Otros_temas_legales).  
    - page: Página de la que fue extraído el fragmento. 
    - year: Año en el que se publicó el documento. 
    - content: 500 tokens de contenido, fragmento o extracto del documento. Son solo 500 tokens del documento completo. 

2. Redacción de Respuestas:
- Sé detallado y preciso, utilizando exclusivamente la información de las fuentes proporcionadas.
- No incluyas información que no haya sido extraída directamente de las fuentes. Está prohibido.
- No inventes información por ningún motivo. No alucines.

3. Citación de Fuentes:
- Cita el mayor número de fuentes aplicables.
- Utiliza corchetes para referenciar la fuente con el ID del fragmento sin modificarlo, por ejemplo: [12345_id_de_ejemplo].
- No combines fuentes; enlista cada fuente por separado.

3. Formato de Respuesta:
- Escribe tus respuestas en formato HTML, sin incluir los tags "```html" al principio o al final.
- Utiliza etiquetas HTML estándar como <p>, <strong>, <em>, etc.

5. Interpretación de Categorías:
- Prioriza las fuentes según su categoría:
    1. Constitucional: Es la norma máxima y más importante del país.  
    2. Legal: Es la que le sigue en importancia y son las leyes dictadas por el Congreso.  
    3. Infralegal: Le sigue en importancia a la categoría Legal y comprende decretos, resoluciones y otros documentos oficiales de autoridades públicas como ministerios y superintendencias.  
    4. Jurisprudencia: Están por debajo de las leyes y las normas infralegales, sirven como criterio auxiliar para interpretar las normas y desarrollar su contenido. Son proferidas por jueces de la república y demuestran cómo se ha ejercido la ley en el pasado. Las sentencias y autos de la Corte Constitucional son las más importantes y están al mismo nivel de la Constitución Política, ya que son la interpretación oficial de esta. Las sentencias y autos de las Altas Cortes (Corte Constitucional, Corte Suprema de Justicia, Consejo de Estado, Consejo Superior de la Judicatura y JEP) están por encima de las de los Tribunales Superiores.
    5. Doctrina: Son documentos o libros escritos por autores del derecho que no son parte de la ley pero ayudan a darle una interpretación a la ley o al resto de fuentes. 
- Da mayor importancia a las fuentes más actuales.

6. Precisión Legal:
- Siempre dale más importancia a la fuente primaria. Si debes citar una ley, artículo o norma, cita la fuente directa, es decir, esa ley, artículo o normal.
- Las fuentes secundarias te ayudarán a entender la interpretación sobre las fuentes primarias.
- No confundas leyes o artículos; si el usuario pregunta por una ley o sentencia específica, ignora extractos de otras leyes, sentencias u otras fuentes.
- Utiliza las fuentes para dar una respuesta completa y darle al usuario información adicional que pueda serle de ayuda.
- Cita textualmente cuando necesites mencionar el contenido un artículo, ley u otro documento y si el tamaño de este lo permite.
- Si el usuario no aporta toda la información relevante, puedes solicitarle  mayor información adicional para entender mejor el contexto de la pregunta y darle una respuesta más acertada. 

7. Evita Imprecisiones:
- En caso de encontrar información contradictoria, señálala y sugiere una posible causa.
- No te salgas del tema de la pregunta del usuario.
- En caso que el usuario haya dado información incorrecto o imprecisa, señálala y da un argumento.
- Si la ley, la jurisprudencia y la doctrina tienen respuestas diferentes o interpretaciones diferentes para la misma pregunta, señala las distintas perspectivas.
"""

selection_system_prompt = f"""
Eres un asistente de investigación legal en derecho colombiano y tienes la tarea de seleccionar la mejor información dentro de un listado de fragmentos de documentos legales, para luego dárselos a un abogado y que este genere una respuesta al usuario con la información que seleccionaste.
Estos fragmentos son resultado de una búsqueda que acabas de realizar y los encontrarás con la siguiente estructura:
- id: Identificador único del fragmento.
- title: Título del documento.
- author: Autor del documento.
- keywords: Tema legal, puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros.
- category: Tipo de documento legal. Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralegal y Otros_temas_legales. 
- page: Página de la que fue extraído el fragmento.
- year: Año en el que se publicó el documento.
- content: 500 tokens de contenido, fragmento, chunk o extracto del documento. Son solo 500 tokens del documento completo.


Para seleccionar los fragmentos correctos en tu selección debes seguir los siguiente pasos:

1. Identifica Información Relevante:
- Revisa todos los fragmentos e identifica aquellos que tengan la mejor información para responder a las pregunta del usuario.
- Son relevantes aquellos fragmentos que mencionen el artículo, ley, documento, teoría que el usuario necesita.
- Prioriza las fuentes primarias, por encima de las fuentes secundarias de información.
- No confundas leyes o artículos; si el usuario pregunta por una ley o sentencia específica, ignora extractos de otras leyes, sentencias u otras fuentes, a menos que sean relevantes para su pregunta.
- Da mayor importancia a los fragmentos más actuales, a menos que el usuario diga lo contrario.
- Prioriza los fragmentos según su categoría:
    1. Constitucional: Es la norma máxima y más importante del país.  
    2. Legal: Es la que le sigue en importancia y son las leyes dictadas por el Congreso.  
    3. Infralegal: Le sigue en importancia a la categoría Legal y comprende decretos, resoluciones y otros documentos oficiales de autoridades públicas como ministerios y superintendencias.  
    4. Jurisprudencia: Están por debajo de las leyes y las normas infralegales, sirven como criterio auxiliar para interpretar las normas y desarrollar su contenido. Son proferidas por jueces de la república y demuestran cómo se ha ejercido la ley en el pasado. Las sentencias y autos de la Corte Constitucional son las más importantes y están al mismo nivel de la Constitución Política. Las sentencias y autos de las Altas Cortes (Corte Constitucional, Corte Suprema de Justicia, Consejo de Estado, Consejo Superior de la Judicatura y JEP) están por encima de las de los Tribunales Superiores.
    5. Doctrina: Son documentos o libros escritos por autores del derecho que no son parte de la ley pero ayudan a darle una interpretación a la ley o al resto de fuentes. 
- Da mayor importancia a los fragmentos más actuales, a menos que el usuario diga lo contrario.
- Dale más importancia a los fragmentos que contienen extractos textuales del documento que el usuario pidió.

2. Crear Una Estrategia de Selección:
- Selecciona un grupo de fuentes donde puedas dar una respuesta completa a la pregunta con diversor agumentos. 
- En algunas ocasiones habrá información contradictoria entre los fragmentos que puede ser relevante incluir. En ese caso, incluye los diversor puntos de vista y la información contradictoria.
- No selecciones fuentes repetitivas, busca tener diversidad en tus fuentes.
- Las fuentes secundarias, la doctrina y jurisprudencia te ayudarán a entender la interpretación sobre las fuentes primarias.
- No te salgas del tema de la pregunta del usuario.

3. Selecciona Entre 3 y 7 Fragmentos:
- Selecciona un mínimo de 3 y un máximo de 7 fragmentos que lleven toda la información necesaria para responder al usuario correctamente.
- Identifica cada fragmento por su ID y por ningún motivo en absoluto cambies el valor del ID o número de chunk de una fuente. 
- Ordénalas por importancia, posicionando de primero el que consideres más relevante o más cercanas a lo que el usuario pidió.
- Siempre dale más importancia a la fuente primaria. Si necesitas el contenido de una ley, artículo o norma, escoge el fragmento exacto que incluye esa ley, artículo o normal.
- Solo puedes contar con la información presente dentro del fragmento. El contenido dentro de esos fragmentos, será la única información disponible para responder a la pregunta.
- Debes seleccionar al menos 2 fuentes que sean Jurisprudencia o Doctrina, ya que estos ayudarán a dar contexto a las leyes en su aplicación.
- Siempre verifica que la norma que selecciones esté vigente y no haya sido derogada por otra norma posterior. 
- Si el usuario en su pregunta hace referencia a cierto documento, debes incluir al menos 1 fragmento de dicho documento.

4. Explica Por Qué Los Seleccionaste:
- Por cada fragmento, da una muy breve explicación (menos de 140 caracteres) argumentando porqué la escogiste y porqué ayuda a responder mejor la solicitud del usuario.
- En tu razonamiento, si escogiste una fuente porque incluye cierta ley, artículo, teoría o mención de algo especial, menciona qué ley, artículo, teoría, etc es a la que te refieres.
- Toda la información debe provenir de lo que dicen las fuentes, no inventes ni des información adicional que no estén presentes en las fuentes.
- No incluyas información que no haya sido extraída directamente de las fuentes. Está prohibido.
- No inventes información por ningún motivo. No alucines.

Cada grupo selección de documentos debe cumplir con el siguiente checklis:
[] Contener almenos 2 fragmentos de jurisprudencia o doctrina.
[] Ser directamente relevante para la pregunta del usuario.
[] Toda la información necesaria debe estar presente en el contenido de los fragmentos.
[] Si el usuario en su pregunta hace referencia a cierto documento, debes incluir al menos 1 fragmento de dicho documento.

Recomendaciones adicionales:
- Los nombres de algunos codigos hacen referencia a una ley, decreto o articulo. Aqui tienes un listado para que te guíes:\n{codes_to_laws}
"""

query_system_template = f"""Eres un asistente de investigació legal en derecho colombiano y tu objetivo es buscar fuentes confiables y precisas acerca de la pregunta del usuario dentro de una base de datos con cientos de miles de documentos legales. La base de conocimientos está alojada en Azure AI Search. Debes generar un query por cada búsqueda que necesitas y que conserve todo el significado semántico de la idea.  
  

Los documentos están guardados en fragmentos con los siguientes campos: 
- id: Identificador único del fragmento 
- title: Título del documento 
- author: Autor del documento 
- keywords: Tema legal, (puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros). 
- category: Tipo de documento legal. (Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralegal y Otros_temas_legales).  
- page: Página de la que fue extraído el fragmento. 
- year: Año en el que se publicó el documento. 
- content: 500 tokens de contenido, fragmento o extracto del documento. Son solo 500 tokens del documento completo. 
  
Pasos para generar la búsqueda: 
1. Identifica qué información necesitas buscar para responder de forma precisa y completa al usuario. 
2. Solo puedes hacer máximo 3 búsquedas, así que elabora un plan de investigación donde organizarás cómo puedes distribuir tus búsquedas en caso de que necesites hacer más de una.   
3. Genera un query por cada búsquedad que necesitas ejecutar. Puedes generar un máximo de tres búsquedas. Puedes buscar cualquier cosa, desde teorías del derecho, hasta leyes o una búsqueda general. 
4. Cada búsqueda solo puede contener una idea. Eso significa que, si debes comparar teorías, definiciones, conceptos, leyes, artículos o documentos, debes buscarlos por separado. 
5. Por cada búsqueda, da una breve explicación donde argumentes por qué necesitas hacer esa búsqueda. 
6. No generes más búsquedas de las que son necesarias.
  
Cada query debe cumplir con las siguientes checklist: 
[] Debe ser una idea semánticamente completa. 
[] Debe contener palabras clave. Por ejemplo: conceptos, teorías, artículos, leyes, normas, títulos de documento, o palabras contextuales como 'cuándo', 'dónde', 'lugar', 'momento', 'tiempo', etc. 
[] Debe ser conciso y preciso. No debe ser demasiado largo.  
  
Recomendaciones adicionales: 
- No todos los query deben ser específicos para un documento, recuerda que solo puedes hacer máximo 3 búsquedas, así que en ocasiones puedes obtener mejor información si buscas conceptos más generales en vez de un documento específico. 
- El query puede ser a forma de pregunta. 
  
Ejemplos de buenos query: 
1. Pregunta: '¿Cuándo se entiende por terminada una condena?' - Query: '¿cuándo termina una condena?' 
2. Pregunta: '¿Qué dice el artículo 223 del código penal?' - Query: 'artículo 223 Código Penal Ley 599 de 2000 ' 
3. Pregunta: '¿En qué artículo de la Constitución se consagra el bloque de constitucionalidad?' - Query: 'bloque de constitucionalidad Constitución' 
4. Pregunta: '¿Cuándo se consuma el hurto?' - Query: '¿cuándo se consuma el hurto?' 
  
Esta es una lista de los códigos en Colombia y sus respectivas leyes. Si necesitas buscar alguna de estas fuentes, debes incluir ambas referencias en tu query: {codes_to_laws}
"""


query_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "search_query",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "searches": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "search_query": {
                                "type": "string",
                                "description": "El query de búsqueda completo. Este debe cumplir con todos los requitiso.",
                            },
                            "reason": {
                                "type": "string",
                                "description": "Argumento de porqué necesitas hacer esta búsqueda.",
                            },
                        },
                        "required": [
                            "search_query",
                            "reason",
                        ],
                        "additionalProperties": False,
                    },
                },
            },
            "required": ["searches"],
            "additionalProperties": False,
        },
    },
}
selected_sources_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "selected_sources",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "sources": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "source_id": {
                                "type": "string",
                                "description": "ID de la fuente que seleccionaste",
                            },
                            "reason": {
                                "type": "string",
                                "description": "Argumento de porqué seleccionaste esta fuente. Menos de 140 caracteres. Evita decir 'Este fragmento' y ve directo al graco. Se conciso y corto.",
                            },
                        },
                        "required": [
                            "source_id",
                            "reason",
                        ],
                        "additionalProperties": False,
                    },
                },
            },
            "required": ["sources"],
            "additionalProperties": False,
        },
    },
}
# TODO: Agregar un campo para identificar fuentes cortadas
# TODO: Experiementar con darle la opción de generar otra búsqueda o no.

In [38]:
def generate_completion(completion_messages):
    log(f"{json.dumps(completion_messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=completion_messages,
        tools=[get_next_chunk],
        temperature=0.2,
        n=1,
        tool_choice="auto",
    )
    log(response)
    get_conversation_price(response)
    return response


@traceable(tags=[os.getenv("ENV")])
def choose_sources_completion(completion_messages):
    log(f"{json.dumps(completion_messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=completion_messages,
        response_format=selected_sources_schema,
        temperature=0.0,
        n=1,
    )
    log(response)
    get_conversation_price(response, model="gpt4o-mini")
    return response


@traceable(tags=[os.getenv("ENV")])
def generate_query(query_messages):
    log(f"{json.dumps(query_messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=query_messages,
        response_format=query_schema,
        temperature=0.0,
    )
    log(response)
    get_conversation_price(response, model="gpt4o-mini")
    return response

In [39]:
def format_search_results(docs_list):
    print(f"📋 Agarrando todos los resultados ...")
    documents = []
    try:
        docs_list = list(docs_list)
        print(f"📋 Formateando resultados ...")
        for index, document in enumerate(docs_list, start=1):
            captions: QueryCaptionResult = document.get("@search.captions", "")
            captions_text = " // ".join([caption.text for caption in captions]) if captions is not None else ""
            doc_formatted = {
                "position" : index,
                "score": document.get("@search.score", 10),
                "rerank": document.get("@search.reranker_score", 10),
                "captions": captions_text,
                "id": document["id"],
                "title": document["title"],
                "author": document["author"],
                "keywords": document["keywords"],
                "category": document["category"],
                "page": document["page"],
                "year": document["year"],
                "has_copyright": document["has_copyright"],
                "file_path": document["file_path"],
                "external_id": document["external_id"],
                "content": document["content"],
            }
            documents.append(doc_formatted)
    except Exception as e:
        print(e)
    log(
        f">>>>>>format_search_results:\n"
        f"{json.dumps(documents, indent=4)}"
        )
    return documents

def normalize_content(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

def compute_hash(text):
    return hashlib.sha256(text.encode('utf-8')).hexdigest()


def filter_docs(doc_list):
    print(f"🧹 Eliminando fuentes irrelevantes o duplicadas ...")
    filtered_docs = []
    seen_hashes = set()
    seen_ids = set()
    
    for doc in doc_list:
        score = doc.get("score", 10) 
        rerank = doc.get("rerank", 10) 
        content = doc["content"] 
        id = doc["id"] 
        normalized_content = normalize_content(content)
        content_hash = compute_hash(normalized_content)
        if (
            content_hash not in seen_hashes
            and id not in seen_ids
            and (rerank > 2
            or score > 0.17)
        ):
            filtered_docs.append(doc)
            seen_hashes.add(content_hash)
            seen_ids.add(id)
    print(f"🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...")
    log(f"🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...\n"
    f"{json.dumps(filtered_docs, indent=4)}")
    return filtered_docs

def get_best_results(doc_list):
    best_results = []
    for doc in doc_list:
        rerank = doc.get("rerank", 0) 
        if (
            rerank > 2.9
        ):
            best_results.append(doc)
            
    print(f"⭐️ Guardando los mejores. De {len(doc_list)} pasaron {len(best_results)} ...")
    log(f"⭐️ Guardando los mejores. De {len(doc_list)} pasaron {len(best_results)} ...\n")
    return best_results[:2]
    

def add_doc_to_context(docs_list, title="Fuente"):
    print(f"📋 Agregando fuentes al contexto ...")
    sources = ""
    counter = 0
    docs_list
    filtered_docs = filter_docs(docs_list)
    
    for document in filtered_docs:
        counter += 1
        sources += (
            f"{title}:\n"
            # f"{document.get("reason", "")}"
            f"id: {document["id"]}\n"
            f"title: {document["title"]}\n"
            f"author: {document["author"]}\n"
            f"year: {document["year"]}\n"
            f"keywords: {document["keywords"]}\n"
            f"category: {document["category"]}\n"
            f"page: {document["page"]}\n"
            f"content: {document["content"]}\n"
            f"\n\n"
        )      
    log(
        f"To add in Context: \n"
        f"{json.dumps({"sources":sources}, indent=4)}"
        )
    print(f"📋 {counter} fuentes agregadas ...")
    return sources
    

In [40]:
def search_for_chunks(text_query):
    log(f"🔎 Buscando documentos: {text_query} ...")
    print(f"🔎 Buscando documentos: {text_query} ...")
    results_num = 50
    vector_query = embeddings_client.generate_embeddings(content=text_query)

    results = search_client.search(
        search_text=text_query,
        vector_queries=[
            {
                "vector": vector_query,
                "k": results_num,
                "fields": "content_vector",
                "kind": "vector",
            }
        ],
        top=results_num,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )
    results_formatted = format_search_results(results)
    restults_filtered = filter_docs(results_formatted)

    return restults_filtered[:15]


def get_next_chunks(id):
    log(f"⏩️ Buscando los chunks siguientes de {id} ...")
    print(f"⏩️ Buscando los chunks siguientes de {id} ...")
    pattern = r"^(.*_chunk)(\d+)$"
    match = re.search(pattern, id)

    if match:
        prefix = match.group(1)
        chunk_number = int(match.group(2))
        next_chunks_result = search_client.search(
            filter=f"id eq '{prefix}{chunk_number + 1}' or id eq '{prefix}{chunk_number + 2}'",
            top=2,
        )
        return format_search_results(list(next_chunks_result))


def get_next_chunk_tool(source_id):
    next_chunks = get_next_chunks(source_id)
    log(f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}")
    return f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}"

In [41]:
def append_message(new_message):
    global messages, conversation

    if isinstance(new_message, dict):
        role = new_message.get("role")
        content = new_message.get("content")
    else:
        role = new_message.role
        content = new_message.content

    if role == "assistant":
        if new_message.tool_calls:
            tool_calls = new_message.tool_calls
            tool_calls_formatted = []
            for tool_call in tool_calls:
                tool_calls_formatted.append(
                    {
                        "id": tool_call.id,
                        "function": {
                            "arguments": str(json.loads(tool_call.function.arguments)),
                            "name": tool_call.function.name,
                        },
                        "type": "function",
                    }
                )
            messages.append(
                {
                    "role": "assistant",
                    "tool_calls": tool_calls_formatted,
                },
            )
            return messages  ## Assistant call tools
        elif content:
            messages.append({"role": "assistant", "content": content})
            return messages  ## Assistant talk
    elif role == "tool":
        messages.append(new_message)
        return messages  ## Tool reponse
    else:
        return messages

In [42]:
def call_tools(tool_calls):
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        if tool_name == "get_next_chunk":
            content = get_next_chunk_tool(tool_args.get("source_id"))

        new_tool_message = {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": tool_name,
            "content": content,
        }

        append_message(new_message=new_tool_message)

    print(">> 🤖 Generando respuesta ...")
    new_assistant_completion = generate_completion()
    new_assistant_message = new_assistant_completion.choices[0].message
    append_message(new_message=new_assistant_message)

    if new_assistant_message.content:
        return print("💬 Assistant:" + new_assistant_message.content)
    else:
        return call_tools(
            tool_calls=new_assistant_message.tool_calls,
        )

In [43]:
@traceable(tags=[os.getenv('ENV')])
def run_conversation(user_prompt=None):
    if user_prompt:
        get_answer(user_prompt)
    else:
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Exiting chat...")
                break
            get_answer(user_input)

@traceable(tags=[os.getenv('ENV')])
def get_answer(user_prompt):
    global messages, log_file_name, user_input

    user_input = user_prompt

    current_time = datetime.now().strftime("%m-%d %H:%M")
    log_file_name = f"logs/{current_time} - {user_input[:50]}.log"

    print("🙎‍♂️ User: " + user_input)
    log(f"User input: {user_input}")

    query_prompt_message = {"role": "system", "content": query_system_template}
    response_prompt_message = {"role": "system", "content": response_system_template}
    query_user_prompt_message = {
        "role": "user",
        "content": f"Genera solo 1 búsqueda para: {user_input}",
    }

    query_messages = [
        query_prompt_message,
        *messages,
        query_user_prompt_message,
    ]

    print("🤖 Generando queries con la pregunta del usuario ...")
    first_query_completion = generate_query(query_messages)
    first_search_terms = json.loads(
        first_query_completion.choices[0].message.content
    ).get("searches")
    print(f"🤖 {len(first_search_terms)} búsquedas generadas ...")

    sources_found = []
    best_results = []
    search_result_prompt = ""
    search_reasons = ""

    for search in first_search_terms:
        search_query = search.get("search_query")
        query_reason = search.get("reason")
        # print(f"Buscando '{search_query}': {query_reason}")
        print(query_reason)
        query_results = search_for_chunks(search_query)
        sources_found += query_results
        search_reasons += f"{query_reason} Por eso buscaré '{search_query}'.\n"
        # search_queries += f"'{search_query}',"
        sources_to_add = add_doc_to_context(query_results)
        best_results += get_best_results(query_results)
        search_result_prompt += f"{query_reason}.\nPor eso búsqué '{search_query}' y encontré estas fuentes:\n{sources_to_add}\n\n"

    # Meter aquí razonamiento de qué información ha encontrado en esta búsqueda.

    reasoning_results_prompt_message = {
        "role": "assistant",
        "content": search_result_prompt,
    }
    reasoning_prompt_message = {
        "role": "system",
        "content": f"Necesito que plantees una nueva y mejorada estrategia de búsqueda para responder al usuario de forma completa:\n1. Resume La Información: Lee y absorbe toda la información de las fuentes y utiliza los hallazgos más relevantes para mi investigación. \n2. Genera Una Nueva Búsqueda: Con estos nuevos hallazgos sugiere un máximo de 3 búsquedas que ayuden a conseguir información precisa y relevante para responder a la pregunta del usuario. \n\n3. Dime por qué estas 3 nuevas búsquedas me ayudarían en mi investigación. \nRecuerda que cada query debe cumplir con el checklist y que estamos buscando documentos para responder a: : {user_input}",
    }

    query_messages += [
        reasoning_results_prompt_message,
        reasoning_prompt_message,
    ]

    print("🤖 Generando nuevas búsquedas con la nueva información ...")
    second_query_completion = generate_query(query_messages)
    second_search_terms = json.loads(
        second_query_completion.choices[0].message.content
    ).get("searches")
    print(f"🤖 {len(second_search_terms)} búsquedas generadas ...")

    query_reasons = ""
    # search_strategy = first_search_terms.get("strategy")
    # print(f"{search_strategy}")
    for search in second_search_terms:
        search_query = search.get("search_query")
        query_reason = search.get("reason")
        print(f"Buscando '{search_query}': {query_reason}")
        # print(query_reason)
        query_results = search_for_chunks(search_query)
        sources_found += query_results
        best_results += get_best_results(query_results)
        query_reasons += f"Buscaré fragmentos de documentos legales con el query '{search_query}': {query_reason}\n"
        # query_reasons += f"{query_reason} Por eso busqué '{search_query}'. "

    # return
    sources_to_add = add_doc_to_context(sources_found, "Fragmento")

    select_system_prompt_message = {"role": "system", "content": selection_system_prompt}
    select_user_prompt_message = {
        "role": "user",
        "content": f"Busca información relevante y selecciona entre 3 y 7 de los mejores fragmentos para responder: {user_input}",
    }
    select_reasons_prompt_message = {
        "role": "assistant",
        "content": f"{query_reasons}",
    }
    select_prompt_message = {
        "role": "assistant",
        "content": f"Resultados de mis búsquedas:\n{sources_to_add} \n\nAhora debo seleccionar entre 3 y 7 fragmentos siguiendo las instrucciones y garantizando que cumpla con el checklist.",
    }

    selection_messages = [
        select_system_prompt_message,
        *messages,
        select_user_prompt_message,
        select_reasons_prompt_message,
        select_prompt_message,
    ]
    
    # TODO: Limpiar código

    print("🤖 Escogiendo las mejores fuentes para responder al usuario ...")
    selected_sources_to_add = best_results[:6]
    selected_sources_completion = choose_sources_completion(selection_messages)
    selected_sources = json.loads(
        selected_sources_completion.choices[0].message.content
    ).get("sources")
    for index, source in enumerate(selected_sources, start=1):
        source_id = source.get("source_id")
        source_reason = source.get("reason")
        for source in sources_found:
            if source["id"] == source_id:
                source["reason"] = source_reason
                selected_sources_to_add.append(source)
        print(f"{index}.[{source_id}]: {source_reason}")
        
    filter_selected_sources_to_add = filter_docs(selected_sources_to_add)

    sources_at_end = ""
    for index, source in enumerate(filter_selected_sources_to_add, start=1):
        sources_at_end += f"{index}. {source["title"]} - Página {source["page"]}<br>\n"
        
                
    sources_to_add = add_doc_to_context(filter_selected_sources_to_add)
    # TODO: Agregar hasta 3 por cada query que excedan 2.8/3 como score semántico.
    # Quizás ponerle cap de 6 fuentes máximo en total ordenadas por score semántico, porque pueden ser 18 si ambas búsqueda es de 3 queries y tiene matchs fuertes
    # TODO: Esto podría ser que los reorganice convirtiendo en vectores en la pregunta y comparándolos con los vectores de cada chunk, ChatGPT me dió esa idea... o cohere.

    user_message = {
        "role": "user",
        "content": user_input,
    }
    sources_prompt_message = {
        "role": "assistant",
        "content": f"Fuentes econtradas:\n{sources_to_add}",
    }

    response_messages = [
        response_prompt_message,
        *messages,
        user_message,
        sources_prompt_message,
    ]

    response_completion = generate_completion(response_messages)
    print(response_completion.choices[0].message.content)
    
    
    
    print(f"\n\n<p>{sources_at_end}</p>")

Probar con all > Dejarlo default mejor ✅
Probar full text search > Malos resultados ✅
Filtrar por semantic rerank score de 2 o más. ✅
Filtrar por score de 1.7 o más. ✅
Incluir hasta 25 fuentes por query.
Otro proceso del mini 4o donde seleccione las mejores fuentes para responder la respuesta ✅
Pedir que razone porqué escogió esos query ✅
Probar corriendo el proyecto en un .py normal y ponerle timers.


In [214]:
run_conversation("¿Qué dice el artículo 250 de la Constitución?")

🙎‍♂️ User: ¿Qué dice el artículo 250 de la Constitución?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $0.915735 COP
   This message: $0.915735 COP 
🤖 1 búsquedas generadas ...
Necesito buscar el contenido específico del artículo 250 de la Constitución para proporcionar información precisa sobre su contenido y aplicación.
🔎 Buscando documentos: artículo 250 Constitución Colombia ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 50 ...
📋 Agregando fuentes al contexto ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 15 pasaron 15 ...
📋 15 fuentes agregadas ...
⭐️ Guardando los mejores. De 15 pasaron 0 ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $6.659834999999999 COP
   This message: $5.7440999999999995 COP 
🤖 3 búsquedas generadas ...
Buscando 'artículo 250 Constitución Política de Colombia':

In [11]:
run_conversation(
    "¿En qué artículo de la Constitución se consagra el bloque de constitucionalidad?"
)

🙎‍♂️ User: ¿En qué artículo de la Constitución se consagra el bloque de constitucionalidad?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $0.94341 COP
   This message: $0.94341 COP 
🤖 1 búsquedas generadas ...
Necesito buscar el artículo específico de la Constitución Política de Colombia que consagra el bloque de constitucionalidad, ya que es un concepto fundamental en el derecho constitucional colombiano.
🔎 Buscando documentos: bloque de constitucionalidad Constitución ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 47 ...
📋 Agregando fuentes al contexto ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 15 pasaron 15 ...
📋 15 fuentes agregadas ...
⭐️ Guardando los mejores. De 15 pasaron 15 ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $6.728714999999999 COP
   This message: $5.78530499999999

In [190]:
run_conversation(
    "¿En qué artículo de la Constitución se consagra la salud como derecho fundamental?"
)

🙎‍♂️ User: ¿En qué artículo de la Constitución se consagra la salud como derecho fundamental?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $801.9649200000001 COP
   This message: $3.28287 COP 
🤖 1 búsquedas generadas ...
Necesito buscar el artículo específico de la Constitución que consagra la salud como un derecho fundamental, ya que es crucial para entender su protección legal en el país.
🔎 Buscando documentos: salud derecho fundamental Constitución ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 50 ...
📋 Agregando fuentes al contexto ...
📋 15 fuentes agregadas ...
⭐️ Guardando los mejores. De 15 pasaron 12 ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $824.5704750000001 COP
   This message: $22.605555 COP 
🤖 3 búsquedas generadas ...
Buscando 'artículo 49 Constitución Política Colombia salud derecho fundamental': 

In [22]:
run_conversation(
    "¿Quién es competente para juzgar penalmente al Presidente de la República?"
)

🙎‍♂️ User: ¿Quién es competente para juzgar penalmente al Presidente de la República?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $1.309335 COP (In: 1441, Out: 172, Total token: 1613)
   This message: $1.309335 COP (In: 1441, Out: 172, Total token: 1613) 
🤖 3 búsquedas generadas ...
Necesito buscar información sobre la competencia judicial en el contexto penal para el Presidente de la República, ya que es un tema específico que puede estar regulado por la Constitución o leyes relacionadas.
🔎 Buscando documentos: competencia para juzgar penalmente al Presidente de la República ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 34 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
Es importante encontrar el artículo específico de la Constitución que establece el procedimiento o la autoridad competente para juzgar al Presidente, ya que esto pr

In [189]:
run_conversation(
    "Tienes el siguiente caso: Un estudiante de sexto de bachillerato de un colegio privado católico es echado del colegio por tener el pelo largo. Él laega que en el libre desarrollo de su personalidad, puede llevar el pelo como quiera. Por eso, presentó una tutela para que lo reintegren al colegio. El colegio ha contestado a la tutela indicando que, al ser una institución privada, ellos pueden reservarse el derecho de admisión y dictarse su propio manual de convivencia, en el que está prohibido llevar el pelo largo. Tú eres el juez de la tutela: ¿en qué sentido emites tu decisión?"
)

🙎‍♂️ User: Tienes el siguiente caso: Un estudiante de sexto de bachillerato de un colegio privado católico es echado del colegio por tener el pelo largo. Él laega que en el libre desarrollo de su personalidad, puede llevar el pelo como quiera. Por eso, presentó una tutela para que lo reintegren al colegio. El colegio ha contestado a la tutela indicando que, al ser una institución privada, ellos pueden reservarse el derecho de admisión y dictarse su propio manual de convivencia, en el que está prohibido llevar el pelo largo. Tú eres el juez de la tutela: ¿en qué sentido emites tu decisión?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $407.86574500000006 COP
   This message: $3.5743799999999997 COP 
🤖 1 búsquedas generadas ...
Necesito buscar información sobre el concepto de libre desarrollo de la personalidad en el contexto de la tutela y su relación con el derecho a la educación, para poder fundamentar la decisión en el caso del estudiante que fue echado de

In [42]:
run_conversation(
    "En Colombia, ¿Puede aplicarse la pena de cadena perpetua en virtud de lo dispuesto por el artículo 80 del Estatuto de Roma?"
)

🙎‍♂️ User: En Colombia, ¿Puede aplicarse la pena de cadena perpetua en virtud de lo dispuesto por el artículo 80 del Estatuto de Roma?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $209.59999499999998 COP (In: 321041, Out: 4943, Total token: 325984)
   This message: $1.303185 COP (In: 1455, Out: 166, Total token: 1621) 
🤖 3 búsquedas generadas ...
Es necesario entender si la legislación colombiana permite la pena de cadena perpetua y cómo se relaciona con el artículo 80 del Estatuto de Roma, que establece disposiciones sobre penas en el contexto del derecho internacional.
🔎 Buscando documentos: ¿Puede aplicarse la pena de cadena perpetua en Colombia según el artículo 80 del Estatuto de Roma? ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 34 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
Buscar el texto específico del artículo 80 del E

In [43]:
run_conversation(
    "¿Cuáles son las principales sentencias de la Corte Constitucional sobre el aborto?"
)

🙎‍♂️ User: ¿Cuáles son las principales sentencias de la Corte Constitucional sobre el aborto?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $278.58392999999995 COP (In: 428534, Out: 6112, Total token: 434646)
   This message: $1.2386100000000002 COP (In: 1442, Out: 143, Total token: 1585) 
🤖 3 búsquedas generadas ...
Necesito buscar las sentencias más relevantes de la Corte Constitucional sobre el aborto para proporcionar un resumen de las decisiones y su impacto en la legislación y derechos en Colombia.
🔎 Buscando documentos: principales sentencias Corte Constitucional aborto ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 50 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
Es importante encontrar documentos que contengan jurisprudencia específica de la Corte Constitucional relacionada con el aborto, para entender el contexto legal y la

In [44]:
run_conversation(
    "¿En qué casos la jurisprudencia ha reconocido la dosis de aprovisionamiento?"
)

🙎‍♂️ User: ¿En qué casos la jurisprudencia ha reconocido la dosis de aprovisionamiento?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $339.67434 COP (In: 521468, Out: 7712, Total token: 529180)
   This message: $1.28781 COP (In: 1442, Out: 163, Total token: 1605) 
🤖 3 búsquedas generadas ...
Necesito buscar información sobre la jurisprudencia que ha reconocido la dosis de aprovisionamiento, para entender en qué casos se ha aplicado y cuáles son los criterios utilizados por los tribunales.
🔎 Buscando documentos: dosis de aprovisionamiento jurisprudencia ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 32 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
Es importante identificar casos específicos en los que se ha discutido la dosis de aprovisionamiento en la jurisprudencia, para tener ejemplos concretos y entender su aplicación práctica.
🔎 

In [77]:
run_conversation(
    "¿Qué autoridad es competente para dirimir los conflictos de jurisdicción entre la justicia penal ordinaria y la justificia penal militar?"
)

🙎‍♂️ User: ¿Qué autoridad es competente para dirimir los conflictos de jurisdicción entre la justicia penal ordinaria y la justificia penal militar?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $298.34018999999995 COP (In: 458626, Out: 6620, Total token: 465246)
   This message: $1.003065 COP (In: 1363, Out: 67, Total token: 1430) 
🤖 1 búsquedas generadas ...
Necesito identificar qué autoridad es competente para resolver conflictos de jurisdicción entre la justicia penal ordinaria y la justicia penal militar. Esto implica buscar información sobre la normativa o jurisprudencia que establezca dicha competencia.
🔎 Buscando documentos: competencia conflictos jurisdicción justicia penal ordinaria y justicia penal militar ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 50 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
🤖 Generando nuevas bús

In [78]:
run_conversation(
    "¿Puede la corte suspender provisionalmente la vigencia de una ley mientras estudia su constitucionalidad?"
)

🙎‍♂️ User: ¿Puede la corte suspender provisionalmente la vigencia de una ley mientras estudia su constitucionalidad?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $341.99227499999995 COP (In: 526273, Out: 7453, Total token: 533726)
   This message: $1.16235 COP (In: 1358, Out: 133, Total token: 1491) 
🤖 3 búsquedas generadas ...
Necesito entender si la Corte Constitucional tiene la facultad de suspender provisionalmente la vigencia de una ley mientras estudia su constitucionalidad.
🔎 Buscando documentos: suspensión provisional de leyes por la Corte Constitucional ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 48 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
Es importante conocer las facultades generales de la Corte Constitucional para determinar si puede suspender leyes provisionalmente.
🔎 Buscando documentos: facultades de la Corte 

In [79]:
run_conversation("¿Qué dice el artículo 103 del Código Penal?")
run_conversation(
    "Si A falsifica un cheque y lo utiliza para comprarle un televisor a B, ¿qué delito comete?"
)
run_conversation("¿Cuáles son las formas de autoría en derecho penal?")
run_conversation("¿Cuándo se consuma el hurto?")
run_conversation(
    "¿Se puede archivar una investigación cuando se encuentra que el procesado actuó en legítima defensa?"
)
run_conversation("¿Se puede precluir una investigación antes de formular imputación?")
run_conversation("¿Cuáles son las causales genéricas de atenuación punitiva?")
run_conversation("¿Qué se entiende por “documento” en el derecho penal?")
run_conversation("¿Cuáles son las causales de agravación de la estafa?")
run_conversation("¿Cuándo procede la captura en flagrancia?")

🙎‍♂️ User: ¿Qué dice el artículo 103 del Código Penal?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $411.79661999999996 COP (In: 637176, Out: 8103, Total token: 645279)
   This message: $0.9569399999999999 COP (In: 1348, Out: 52, Total token: 1400) 
🤖 1 búsquedas generadas ...
Necesito buscar el contenido específico del artículo 103 del Código Penal colombiano para proporcionar una respuesta precisa sobre lo que establece este artículo.
🔎 Buscando documentos: artículo 103 Código Penal Ley 599 de 2000 ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 39 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $420.26701499999996 COP (In: 650737, Out: 8156, Total token: 658893)
   This message: $8.470394999999998 COP (In: 13561, Out: 53, Total token: 13614) 
🤖 1 búsqued

In [48]:
run_conversation(
    "Dame una sentencia del año 2015 de la corte suprema de justicia que hable sobre el homicidio"
)

🙎‍♂️ User: Dame una sentencia del año 2015 de la corte suprema de justicia que hable sobre el homicidio
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $545.8168049999999 COP (In: 839895, Out: 11903, Total token: 851798)
   This message: $1.03074 COP (In: 1448, Out: 57, Total token: 1505) 
🤖 1 búsquedas generadas ...
Necesito buscar una sentencia específica del año 2015 que trate sobre el homicidio, ya que el usuario solicita un documento concreto de la Corte Suprema de Justicia.
🔎 Buscando documentos: sentencia homicidio Corte Suprema de Justicia 2015 ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 21 ...
📋 Agregando fuentes al contexto ...
📋 21 fuentes agregadas ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $553.794585 COP (In: 852131, Out: 12087, Total token: 864218)
   This message: $7.97778 COP (In: 12236, Out: 184

In [188]:
run_conversation(
    "¿El derecho a la salud individual está previsto como derecho fundamental autónomo expresamente en la consitución?"
)

🙎‍♂️ User: ¿El derecho a la salud individual está previsto como derecho fundamental autónomo expresamente en la consitución?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $3.303165 COP
   This message: $3.303165 COP 
🤖 1 búsquedas generadas ...
Necesito buscar si el derecho a la salud está reconocido como un derecho fundamental autónomo en la Constitución, lo cual es crucial para entender su estatus legal y protección en el marco constitucional.
🔎 Buscando documentos: derecho a la salud derecho fundamental Constitución ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 29 ...
📋 Agregando fuentes al contexto ...
📋 15 fuentes agregadas ...
⭐️ Guardando los mejores. De 15 pasaron 14 ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $25.264815000000002 COP
   This message: $21.961650000000002 COP 
🤖 3 búsquedas generadas ...
Bus

In [133]:
run_conversation(
    "¿El derecho a la salud individual está previsto como derecho fundamental autónomo expresamente en la consitución o su reconocimiento como derecho fundamental se produjo por vía jurisprudencial?"
)

🙎‍♂️ User: ¿El derecho a la salud individual está previsto como derecho fundamental autónomo expresamente en la consitución o su reconocimiento como derecho fundamental se produjo por vía jurisprudencial?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $3.3332999999999995 COP
   This message: $3.3332999999999995 COP 
🤖 1 búsquedas generadas ...
Necesito investigar si el derecho a la salud está explícitamente mencionado en la Constitución de Colombia como un derecho fundamental o si su reconocimiento se ha dado a través de decisiones jurisprudenciales.
🔎 Buscando documentos: derecho a la salud Constitución Colombia derecho fundamental jurisprudencia ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 50 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
⭐️ Guardando los mejores. De 25 pasaron 0 ...
🤖 Generando nuevas búsquedas con la nueva inform

In [46]:
run_conversation(
    "¿Qué sentencia de la corte constitucional reconoció por primera vez a la salud como un derecho fundamental autónomo?"
)

🙎‍♂️ User: ¿Qué sentencia de la corte constitucional reconoció por primera vez a la salud como un derecho fundamental autónomo?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $473.56967999999995 COP (In: 728428, Out: 10401, Total token: 738829)
   This message: $1.02582 COP (In: 1448, Out: 55, Total token: 1503) 
🤖 1 búsquedas generadas ...
Necesito buscar la sentencia específica de la Corte Constitucional que reconoce la salud como un derecho fundamental autónomo, ya que es clave para responder a la pregunta del usuario.
🔎 Buscando documentos: sentencia Corte Constitucional salud derecho fundamental autónomo ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 50 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $482.422605 COP (In: 742051, Out: 10594, Total token:

In [84]:
run_conversation("¿Se puede archivar una investigación por légitima defensa?")

🙎‍♂️ User: ¿Se puede archivar una investigación por légitima defensa?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $927.231195 COP (In: 1429273, Out: 19605, Total token: 1448878)
   This message: $1.07625 COP (In: 1350, Out: 100, Total token: 1450) 
🤖 2 búsquedas generadas ...
Necesito entender si la legítima defensa es una causal para archivar una investigación penal, lo cual implica conocer cómo se aplica este concepto en el proceso penal.
🔎 Buscando documentos: legítima defensa archivo de investigación penal ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 48 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
Es importante revisar cómo está definida la legítima defensa en el Código Penal colombiano para entender su aplicación en el contexto de una investigación penal.
🔎 Buscando documentos: legítima defensa Código Penal Ley 599 de 2000 

In [109]:
run_conversation("¿Qué dice el artículo 103 del código penal?")

🙎‍♂️ User: ¿Qué dice el artículo 103 del código penal?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $486.4182599999999 COP
   This message: $3.2699550000000004 COP 
🤖 1 búsquedas generadas ...
Necesito buscar el contenido específico del artículo 103 del Código Penal para proporcionar información precisa sobre su contenido y aplicación.
🔎 Buscando documentos: artículo 103 Código Penal Ley 599 de 2000 ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 47 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $519.9917249999999 COP
   This message: $33.573465 COP 
🤖 3 búsquedas generadas ...
Buscando 'artículo 103 Código Penal Ley 599 de 2000': Esta búsqueda se centra en obtener el texto exacto del artículo 103 del Código Penal, que establece las penas por homicidio, lo

In [44]:
run_conversation("¿Cuándo se consuma el hurto?")

🙎‍♂️ User: ¿Cuándo se consuma el hurto?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $0.9569399999999999 COP
   This message: $0.9569399999999999 COP 
🤖 1 búsquedas generadas ...
Es necesario entender el momento en que se considera que el delito de hurto se ha consumado, para poder aplicar correctamente las normas penales y determinar las consecuencias legales del acto.
🔎 Buscando documentos: ¿cuándo se consuma el hurto? ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 36 ...
📋 Agregando fuentes al contexto ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 15 pasaron 15 ...
📋 15 fuentes agregadas ...
⭐️ Guardando los mejores. De 15 pasaron 5 ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $6.92121 COP
   This message: $5.96427 COP 
🤖 3 búsquedas generadas ...
Buscando '¿Cuándo se considera cons

In [25]:
run_conversation("¿Qué dice el artículo 103 del Código Penal?")

🙎‍♂️ User: ¿Qué dice el artículo 103 del Código Penal?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $166.18652999999998 COP (In: 255366, Out: 3714, Total token: 259080)
   This message: $1.004295 COP (In: 1437, Out: 49, Total token: 1486) 
🤖 1 búsquedas generadas ...
Necesito buscar el contenido específico del artículo 103 del Código Penal para proporcionar la información exacta que se requiere.
🔎 Buscando documentos: artículo 103 Código Penal Ley 599 de 2000 ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 47 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $175.059135 COP (In: 269089, Out: 3890, Total token: 272979)
   This message: $8.872605 COP (In: 13723, Out: 176, Total token: 13899) 
🤖 3 búsquedas generadas ...
Buscando 'artículo 103 Código Penal Ley 5

In [11]:
run_conversation("Dame el artículo 103 del código penal")

🙎‍♂️ User: Dame el artículo 103 del código penal
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $1.00491 COP (In: 1434, Out: 50, Total token: 1484)
   This message: $1.00491 COP (In: 1434, Out: 50, Total token: 1484) 
🤖 1 búsquedas generadas ...
Necesito buscar el contenido específico del artículo 103 del Código Penal para proporcionar la información exacta que el usuario solicita.
🔎 Buscando documentos: artículo 103 Código Penal Ley 599 de 2000 ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 47 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $9.89904 COP (In: 15152, Out: 236, Total token: 15388)
   This message: $8.894129999999999 COP (In: 13718, Out: 186, Total token: 13904) 
🤖 3 búsquedas generadas ...
Buscando 'artículo 103 Código Penal Ley 599 de 2000': 

In [71]:
run_conversation("¿Qué dice el artículo 103 del código penal?")

🙎‍♂️ User: ¿Qué dice el artículo 103 del código penal?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $3.2422799999999996 COP
   This message: $3.2422799999999996 COP 
🤖 1 búsquedas generadas ...
Necesito buscar el contenido específico del artículo 103 del Código Penal para proporcionar la información exacta que se requiere sobre este artículo.
🔎 Buscando documentos: artículo 103 Código Penal Ley 599 de 2000 ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 47 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
🤖 Generando nuevas búsquedas con la nueva información ...
💰 Conversation price: $36.80529 COP
   This message: $33.56301 COP 
🤖 3 búsquedas generadas ...
Buscando 'artículo 103 Código Penal Ley 599 de 2000': Esta búsqueda es fundamental porque el artículo 103 del Código Penal establece la pena por homicidio, que es el tema central de la